In [ ]:
from utils import sysinfo

sysinfo()

# Generic dfs processing
Tools and methods that applies to any type of dfs files.

The generic tools are useful for common data processing tasks, where detailed configuration is not necessary. 

* mikeio.generic: methods that read any dfs file and outputs a new dfs file of the same type
    * concat: Concatenates files along the time axis
    * scale: Apply scaling to any dfs file
    * sum: Sum two dfs files
    * diff: Calculate difference between two dfs files
    * extract: Extract timesteps and/or items to a new dfs file

In [2]:
# TODO